# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 56 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.13908


extracting tarball to tmp_2403.13908...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2403.13908/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Found 67 bibliographic references in tmp_2403.13908/ms.bbl.
syntax error in line 214: '=' expected
Retrieving document from  https://arxiv.org/e-print/2403.14106


extracting tarball to tmp_2403.14106... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.13908-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.13908) | **PHANGS-HST: Globular Cluster Systems in 17 Nearby Spiral Galaxies**  |
|| M. Floyd, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-03-22*|
|*Comments*| *21 pages, 12 figures*|
|**Abstract**| We present new catalogs of likely globular clusters (GCs) in 17 nearby spiral galaxies studied as part of the PHANGS-HST Treasury Survey. The galaxies were imaged in five broad-band filters from the near-ultraviolet through the $I$ band. PHANGS-HST has produced catalogs of stellar clusters of all ages by selecting extended sources (from multiple concentration index measurements) followed by morphological classification (centrally concentrated and symmetric or asymmetric, multiple peaks, contaminant) by visually examining the V-band image and separately by a machine-learning algorithm which classified larger samples to reach fainter limits. From both cluster catalogs, we select an initial list of candidate GCs to have $B-V \geq 0.5$ and $V-I \geq 0.73$~mag, then remove likely contaminants (including reddened young clusters, background galaxies misclassified by the neural network, and chance superpositions/blends of stars) after a careful visual inspection. We find that $\approx86$ % of the color-selected candidates classified as spherically symmetric, and $\approx68$ of those classified as centrally concentrated but asymmetric are likely to be GCs. The luminosity functions of the GC candidates in 2 of our 17 galaxies, NGC 628 and NGC 3627, are atypical, and continue to rise at least 1~mag fainter than the expected turnover near $M_V \sim -7.4$. These faint candidate GCs have more extended spatial distributions than their bright counterparts, and may reside in the disk rather than the bulge/halo, similar to faint GCs previously discovered in M101. These faint clusters may be somewhat younger since the age-metallicity degeneracy makes it difficult to determine precise cluster ages from integrated colors once they reach $\approx1$~Gyr. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.14106-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.14106) | **The disappearance of the blue and luminous progenitor of the Type IIn SN  2010jl**  |
|| Z. Niu, N.-C. Sun, <mark>J. Liu</mark> |
|*Appeared on*| *2024-03-22*|
|*Comments*| *Submitted to ApJL; 4 figures; comments are welcome*|
|**Abstract**| Type IIn supernovae (SNe) exhibit narrow hydrogen lines that arise from the strong interaction between ejecta and circumstellar material. It remains poorly understood, however, what progenitor stars give rise to these explosions. In this work, we perform a detailed analysis of the progenitor and environment of the nearby Type IIn SN 2010jl. With newer images taken by the Hubble Space Telescope, we confirm that the previously reported progenitor candidate is a blend of the progenitor itself and a field star cluster in its close vicinity. SN 2010jl has now become much fainter than the progenitor. The progenitor is very blue and luminous with an effective temperature of log $T_{\rm eff}/{\rm K}$=4.26$^{+0.11}_{-0.09}$ and a luminosity of log $L/L_{\odot}$ =6.52$^{+0.20}_{-0.16}$. It is located in a very young star-forming region, but its luminosity is much higher than that expected from the environmental stellar populations. We suggest that the progenitor was in outburst when observed. Its nature and evolutionary history remain to be investigated. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.13908.md
    + _build/html/tmp_2403.13908/./BG_FS.png
    + _build/html/tmp_2403.13908/./CC_plot_showing_separation.png
    + _build/html/tmp_2403.13908/./Good_Glob.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\UTA}{\affiliation{Instituto de Alta Investigación, Universidad de Tarapacá, Casilla 7D, Arica, Chile}}$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\UToledo}{\affiliation{Ritter Astrophysical Research Center, University of Toledo, Toledo, OH 43606, USA}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218 USA}}$
$\newcommand{\Caltech}{\affiliation{TAPIR, California Institute of Technology, Pasadena, CA 91125 USA}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave., Columbus, OH 43210, USA}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\UAlberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\Bonn}{\affiliation{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121, Bonn, Germany}}$
$\newcommand{\AthreeD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\ITA}{\affiliation{Institut für Theoretische Astrophysik, Zentrum für Astronomie der Universität Heidelberg,\ Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affiliation{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\STScIESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\OHHS}{\affiliation{Ottawa Hills High School, 4035 W. Central Ave, Ottawa Hills, OH, 43606}}$
$\newcommand{\UMICHD}{\affiliation{Department of Natural Sciences, University of Michigan-Dearborn, 4901 Evergreen Road, Dearborn, MI 48128, USA}}$
$\newcommand{\UIOWA}{\affiliation{Department of Physics and Astronomy, University of Iowa, Iowa City, Iowa 52242, USA}}$
$\newcommand{\CWRU}{\affiliation{Department of Astronomy, Case Western Reserve University, 10900 Euclid Avenue, Cleveland, OH 44106, USA}}$
$\newcommand{\lea}{\mathrel{<\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand{\gea}{\mathrel{>\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand\rc{#1}$</div>



<div id="title">

# PHANGS-HST: Globular Cluster Systems in 17 Nearby Spiral Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.13908-b31b1b.svg)](https://arxiv.org/abs/2403.13908)<mark>Appeared on: 2024-03-22</mark> -  _21 pages, 12 figures_

</div>
<div id="authors">

Matthew~Floyd, et al.

</div>
<div id="abstract">

**Abstract:** We present new catalogs of likely globular clusters (GCs) in 17 nearby spiral galaxies studied as part of the PHANGS-HST Treasury Survey. The galaxies were imaged in five broad-band filters from the near-ultraviolet through the $I$ band.  PHANGS-HST has produced catalogs of stellar clusters of all ages by selecting extended sources (from multiple concentration index measurements) followed by morphological classification (centrally concentrated and symmetric or asymmetric, multiple peaks, contaminant) by visually examining the V-band image and separately by a machine-learning algorithm which classified larger samples to reach fainter limits. From both cluster catalogs, we select an initial list of candidate GCs to have $B-V \geq 0.5$ and $V-I \geq 0.73$ mag, then remove likely contaminants (including reddened young clusters, background galaxies misclassified by the neural network, and chance superpositions/blends of stars) after a careful visual inspection.We find that $\approx86$ \% of the color-selected candidates classified as spherically symmetric, and 68 \% of those classified as centrally concentrated but asymmetric are likely to be GCs.  The luminosity functions of the GC candidates in 2 of our 17 galaxies, NGC 628 and NGC 3627, are atypical, and continue to rise at least 1 mag fainter than the expected turnover near $M_V \sim -7.4$ . These faint candidate GCs have more extended spatial distributions than their bright counterparts, and may reside in the disk rather than the bulge/halo, similar to faint GCs previously discovered in M101. These faint clusters may be somewhat younger since the age-metallicity degeneracy makes it difficult to determine precise cluster ages from integrated colors once they reach $\approx1$ Gyr.

</div>

<div id="div_fig1">

<img src="tmp_2403.13908/./BG_FS.png" alt="Fig5" width="100%"/>

**Figure 5. -** Color BVI images showing examples of contaminants from our initial color-selection of GC candidates. The left image shows a background galaxy that was included in the machine-learning catalogs. The center panel shows two background galaxies also from the ML-selected catalogs, while the right panel shows a saturated foreground star from the human-classified catalog. All contaminants are removed from the final GC catalogs. (*fig:RG_FGS*)

</div>
<div id="div_fig2">

<img src="tmp_2403.13908/./CC_plot_showing_separation.png" alt="Fig2" width="100%"/>

**Figure 2. -** Color-color diagrams are shown for all human-selected Class 1$+$2 cluster candidates.
        $U-B$ vs $V-I$  is shown on the left and $B-V$ vs $V-I$ on the right.
    The predicted color evolution from the \citet{Bruzual03} stellar population models are shown for solar metallicity (red) and 1/50$\times$solar metallicity (blue). The light blue oval at the upper left shows the 'young cluster locus', the green oval shows the 'middle-aged plume', and the red oval shows the 'old globular-cluster clump'.
    Dashed lines indicate the color cuts we use to select our initial list of GC candidates, B-V $\geq 0.5$ and V-I $\geq$ 0.73. Only those candidates with significant U-band measurements are plotted in the left panel.
    The arrows represent the direction and amount of reddening for $A_V = 1$. (*fig:human12_cc*)

</div>
<div id="div_fig3">

<img src="tmp_2403.13908/./Good_Glob.png" alt="Fig3" width="100%"/>

**Figure 3. -** Diagnostic plots for an ancient globular cluster. The source appears symmetric, round, and has a uniform color in a $BVI$ image (left). The right panel, a $B-V-$ CO image, shows there is no CO emission associated with the cluster. In the color-color diagrams shown in the center panel, the cluster (blue circle) lies close to predictions from the $1/50 \times$solar metallicity SSP model.
    Black star symbols mark ages of 1 Myr, 10 Myr, 100 Myr, 1 Gyr, and 10 Gyr on the solar metallicity track.  A cyan star marks the predicted position for 10 Gyr in the 1/50th solar model. (*fig:good glob*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.13908"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

392  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
